In [ ]:
!pip install kafka-python pyspark

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.13:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.1,org.apache.kafka:kafka-clients:3.5.1 pyspark-shell'


In [ ]:
import findspark
findspark.init('/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
#from pyspark.streaming.kafka import KafkaUtils

# Crear el contexto de Spark
sc = None
try:
    sc = SparkContext(appName="movielens")
except:
    sc = SparkContext.getOrCreate("movielens")
ssc = StreamingContext(sc, 10)  # Intervalo de 10 segundos

# Structured Streaming

Para conectarse con Kafka, se utilia un paradigma diferente llamado **Structured Streaming**.

Documentación: [Spark Streaming + Kafka Integration Guide](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html)

Mas general, documentación sobre Structured Streaming: [Spark Streaming Programming Guide](https://spark.apache.org/docs/latest/streaming-programming-guide.html) sobre todo la sección de [Basic Concepts](https://spark.apache.org/docs/latest/streaming-programming-guide.html#basic-concepts)

In [ ]:

spark = SparkSession.builder \
    .appName("movielens") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.1,org.apache.kafka:kafka-clients:3.5.1") \
    .getOrCreate()

!spark-submit --class movielens --packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.1,org.apache.kafka:kafka-clients:3.5.1 09\ Streaming\ Kafka.ipynb



In [ ]:
schema =  StructType([
            StructField("col1", StringType()),
            StructField("col2", StringType())
            ])

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:2181") \
  .option("subscribe", "test-topic") \
  .load()

value_df = df.select(col("topic"), col("partition"), col("offset"), from_json(col("value").cast("STRING"), schema).alias("values"))

print(value_df)
# Conexión a Kafka
#kafkaStream = KafkaUtils.createStream(ssc, "localhost:2181", "spark-streaming", {"test-topic": 1})

# Extraer solo los mensajes del stream de Kafka
#messages = kafkaStream.map(lambda x: x[1])

In [ ]:
!pip freeze

# Envío de mensajes

Antes de ejecutar el siguiente código, se puede ejecutar el cliente de consola de Kafka para poder enviarle mensajes. Esto se puede realizar a traves del siguiente comando:

```
bin/kafka-console-producer.sh --broker-list localhost:9092 --topic test-topic
```
Y se puede digitar en la consola el mensaje a enviar.

In [ ]:
# Contar las palabras en los mensajes
words = messages.flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Mostrar el conteo de palabras
wordCounts.pprint()

ssc.start()
ssc.awaitTermination()